In [2]:
import pandas as pd
data = pd.read_csv(r"C:\Users\mahmo\Downloads\first inten project.csv")

In [3]:
data.isnull().sum()

Booking_ID                  0
number of adults            0
number of children          0
number of weekend nights    0
number of week nights       0
type of meal                0
car parking space           0
room type                   0
lead time                   0
market segment type         0
repeated                    0
P-C                         0
P-not-C                     0
average price               0
special requests            0
date of reservation         0
booking status              0
dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Booking_ID                36285 non-null  object 
 1   number of adults          36285 non-null  int64  
 2   number of children        36285 non-null  int64  
 3   number of weekend nights  36285 non-null  int64  
 4   number of week nights     36285 non-null  int64  
 5   type of meal              36285 non-null  object 
 6   car parking space         36285 non-null  int64  
 7   room type                 36285 non-null  object 
 8   lead time                 36285 non-null  int64  
 9   market segment type       36285 non-null  object 
 10  repeated                  36285 non-null  int64  
 11  P-C                       36285 non-null  int64  
 12  P-not-C                   36285 non-null  int64  
 13  average price             36285 non-null  float64
 14  specia

In [5]:
data = data.rename(columns={
    'column_name_1': 'number of adults',
    'column_name_2': 'number of children',
    'column_name_3': 'number of weekend nights',
    'column_name_4': 'number of week nights',
    'column_name_5': 'type of meal',
    'column_name_6': 'car parking space',
    'column_name_7': 'room type',
    'column_name_8': 'lead time',
    'column_name_9': 'market segment type',
    'column_name_10': 'average price',
    'column_name_11': 'special requests',
    'column_name_12': 'date of reservation',
    'column_name_13': 'booking status'
})

In [6]:
data = data.drop_duplicates()

In [7]:
import numpy as np
from scipy.stats import zscore

def remove_outliers(df, columns, z_threshold=3):
    """
    Remove outliers from specified columns using z-score method.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    columns (list): List of column names to check for outliers
    z_threshold (float): Z-score threshold for outlier detection (default: 3)
    
    Returns:
    pandas.DataFrame: DataFrame with outliers removed
    dict: Statistics about removed outliers
    """
    initial_rows = len(df)
    stats = {}
    
    # Create a mask that starts with all True values
    combined_mask = np.ones(len(df), dtype=bool)
    
    for column in columns:
        if column in df.columns and df[column].dtype in ['int64', 'float64']:
            z_scores = zscore(df[column], nan_policy='omit')
            column_mask = (z_scores < z_threshold) & (z_scores > -z_threshold)
            combined_mask = combined_mask & column_mask
            
            # Store statistics
            stats[column] = {
                'outliers_removed': np.sum(~column_mask),
                'outlier_percentage': (np.sum(~column_mask) / len(df)) * 100
            }
    
    # Apply the combined mask to the DataFrame
    df_cleaned = df[combined_mask].copy()
    
    # Add summary statistics
    stats['summary'] = {
        'initial_rows': initial_rows,
        'remaining_rows': len(df_cleaned),
        'total_rows_removed': initial_rows - len(df_cleaned),
        'total_percentage_removed': ((initial_rows - len(df_cleaned)) / initial_rows) * 100
    }
    
    return df_cleaned, stats

# Strip whitespace from column names
data.columns = data.columns.str.strip()

# Print original column names
print("Original columns:", data.columns.tolist())

# Identify numeric columns automatically
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("\nNumeric columns for outlier removal:", numeric_columns)

# Remove outliers from numeric columns
data_cleaned, outlier_stats = remove_outliers(data, numeric_columns)

# Print summary statistics
print("\nOutlier Removal Summary:")
print(f"Initial number of rows: {outlier_stats['summary']['initial_rows']}")
print(f"Remaining rows: {outlier_stats['summary']['remaining_rows']}")
print(f"Total rows removed: {outlier_stats['summary']['total_rows_removed']}")
print(f"Percentage of data removed: {outlier_stats['summary']['total_percentage_removed']:.2f}%")

print("\nOutliers removed per column:")
for column in numeric_columns:
    if column in outlier_stats:
        print(f"{column}: {outlier_stats[column]['outliers_removed']} rows "
              f"({outlier_stats[column]['outlier_percentage']:.2f}%)")

# Save cleaned data if needed
# data_cleaned.to_csv('cleaned_data.csv', index=False)

# Display first few rows of cleaned data
print("\nFirst few rows of cleaned data:")
print(data_cleaned.head())

Original columns: ['Booking_ID', 'number of adults', 'number of children', 'number of weekend nights', 'number of week nights', 'type of meal', 'car parking space', 'room type', 'lead time', 'market segment type', 'repeated', 'P-C', 'P-not-C', 'average price', 'special requests', 'date of reservation', 'booking status']

Numeric columns for outlier removal: ['number of adults', 'number of children', 'number of weekend nights', 'number of week nights', 'car parking space', 'lead time', 'repeated', 'P-C', 'P-not-C', 'average price', 'special requests']

Outlier Removal Summary:
Initial number of rows: 36285
Remaining rows: 31883
Total rows removed: 4402
Percentage of data removed: 12.13%

Outliers removed per column:
number of adults: 155 rows (0.43%)
number of children: 1080 rows (2.98%)
number of weekend nights: 184 rows (0.51%)
number of week nights: 324 rows (0.89%)
car parking space: 1124 rows (3.10%)
lead time: 377 rows (1.04%)
repeated: 930 rows (2.56%)
P-C: 140 rows (0.39%)
P-not

In [8]:
data = data.drop(['Booking_ID', 'P-C', 'P-not-C'], axis=1)

In [9]:
data['total_nights'] = data['number of weekend nights'] + data['number of week nights']

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['type of meal'] = le.fit_transform(data['type of meal'])
data['room type'] = le.fit_transform(data['room type'])
data['market segment type'] = le.fit_transform(data['market segment type'])

In [11]:
data = pd.get_dummies(data, columns=['type of meal', 'room type', 'market segment type'], drop_first=True)

In [ ]:
print(data["date of reservation"].dtype)
print(data["date of reservation"].head())

datetime64[ns]
0   2015-10-02
1   2018-11-06
2   2018-02-28
3   2017-05-20
4   2018-04-11
Name: date of reservation, dtype: datetime64[ns]


In [40]:
X_train["date of reservation"] = pd.to_datetime(X_train["date of reservation"], errors="coerce")
X_train["date of reservation"] = X_train["date of reservation"].astype("int64") // 10**9  # Convert to Unix timestamp

In [42]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [44]:
X_test["date of reservation"] = pd.to_datetime(X_test["date of reservation"], errors="coerce")
X_test["date of reservation"] = X_test["date of reservation"].astype("int64") // 10**9

In [45]:
y_pred = model.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.898442882733912
              precision    recall  f1-score   support

    Canceled       0.88      0.80      0.84      2402
Not_Canceled       0.91      0.94      0.93      4855

    accuracy                           0.90      7257
   macro avg       0.89      0.87      0.88      7257
weighted avg       0.90      0.90      0.90      7257

